In [1]:
from multiprocessing import Pool
from tqdm.notebook import tqdm
import numpy as np

from modules.udpipe_preprocessing import preprocess

n_cpus = 24

batch_size = 24_000

In [8]:
def preprocess_batch(batch):
    return [' '.join(preprocess(text)) + '\n' for text in batch]

In [9]:
with open('../LM-experiments_2/joined_news_long_texts_8796201_293713268.txt') as f, open('joined_news_long_texts_udpipe_short.txt', 'w') as fw, Pool(n_cpus) as p, tqdm(total=8796201) as pbar:
    line = f.readline()
    buffer = []
    while line:
        buffer.append(line)
        if len(buffer) == batch_size:
            spl = np.array_split(buffer, 24)
            udpiped = p.map(preprocess_batch, spl)
            for s in udpiped:
                fw.writelines(s)
            buffer = []
            pbar.update(batch_size)
        line = f.readline()
    spl = np.array_split(buffer, 24)
    udpiped = p.map(preprocess_batch, spl)
    for s in udpiped:
        fw.writelines(s)
    pbar.update(len(buffer))

  0%|          | 0/8796201 [00:00<?, ?it/s]

In [10]:
from tqdm.notebook import tqdm
from collections import Counter 

vocab_counter = Counter()

with open('joined_news_long_texts_udpipe_short.txt') as f, tqdm(total=8796201) as pbar:
    line = f.readline()
    while line:
        tokens = line.strip().split()
        vocab_counter += Counter(tokens)
        pbar.update()
        line = f.readline()

  0%|          | 0/8796201 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [11]:
len(vocab_counter)

388267

In [12]:
import pandas as pd

In [15]:
vocab_counter = pd.Series(vocab_counter)

In [19]:
vocab_counter = vocab_counter.sort_values(ascending=False)

In [29]:
vocab_counter[vocab_counter > 10].shape

(53075,)

In [32]:
vocab = vocab_counter[vocab_counter > 10].index.tolist()

In [34]:
with open('vocab.txt', 'w') as f:
    f.write('<S>\n</S>\n<UNK>\n')
    for token in vocab:
        f.write(token)
        f.write('\n')

In [35]:
vocab_counter = vocab_counter[vocab_counter > 10]

In [38]:
vocab_counter[vocab_counter > 50].shape

(20567,)